In [1]:
#Prüfungsaufgabe_SK2
import pandas as pd
from io import BytesIO
from ipywidgets import widgets, VBox, HBox, Output, Layout
from scipy.optimize import minimize

# Funktionen
def berechne_nutzenwerte(angebot, praefmod1, praefmod2):
    
    def berechne_erfuellungsgrad(row, angebot):
        aspiration_level = row['AL_i']
        reservation_level = row['RL_i']
        wert = angebot.get(row['Attribut'], 0)

        # Berechnung des Erfüllungsgrades
        if aspiration_level > reservation_level:
            if wert <= reservation_level:
                erfuellungsgrad = 0
            elif wert >= aspiration_level:
                erfuellungsgrad = 1
            else:
                erfuellungsgrad = wert / (aspiration_level - reservation_level) - (reservation_level / (aspiration_level - reservation_level))
        else: 
            if wert >= reservation_level:
                erfuellungsgrad = 0
            elif wert <= aspiration_level:
                erfuellungsgrad = 1
            else:
                erfuellungsgrad = wert / (aspiration_level - reservation_level) - (reservation_level / (aspiration_level - reservation_level))

        return erfuellungsgrad

    def berechne_nutzen(row, angebot):
        gewichtung = row['W_i']
        erfuellungsgrad = berechne_erfuellungsgrad(row, angebot)
        return erfuellungsgrad * gewichtung


    
    erfuellungsgrad_gehalt = berechne_erfuellungsgrad(praefmod1[praefmod1["Attribut"] == "Gehalt"].iloc[0], angebot)
    if (erfuellungsgrad_gehalt < 0.5):
        praefmod1.loc[praefmod1["Attribut"] == "Gehalt", "W_i"] += 0.1
        praefmod1.loc[praefmod1["Attribut"] == "Urlaubstage", "W_i"] -= 0.05
        praefmod1.loc[praefmod1["Attribut"] == "Zuzahlung (PKW)", "W_i"] -= 0.05

    erfuellungsgrad_urlaub = berechne_erfuellungsgrad(praefmod2[praefmod2["Attribut"] == "Urlaubstage"].iloc[0], angebot)
    if (erfuellungsgrad_urlaub < 0.5):
        praefmod2.loc[praefmod2["Attribut"] == "Gehalt", "W_i"] -= 0.05
        praefmod2.loc[praefmod2["Attribut"] == "Urlaubstage", "W_i"] += 0.1
        praefmod2.loc[praefmod2["Attribut"] == "Zuzahlung (PKW)", "W_i"] -= 0.05


    print(praefmod1)
    print(praefmod2)
    nutzen1 = praefmod1.apply(berechne_nutzen, axis=1, angebot=angebot).sum()
    nutzen2 = praefmod2.apply(berechne_nutzen, axis=1, angebot=angebot).sum()

    def zielfunktion(angebot_values):
        angebot_opt = {
            'Gehalt': angebot_values[0],
            'Urlaubstage': angebot_values[1],
            'Zuzahlung (PKW)': angebot_values[2]
        }
        nutzen1_opt = praefmod1.apply(berechne_nutzen, axis=1, angebot=angebot_opt).sum()
        nutzen2_opt = praefmod2.apply(berechne_nutzen, axis=1, angebot=angebot_opt).sum()
        return -(nutzen1_opt + nutzen2_opt)  # Negativ für Minimierung

    # Gemeinsame Bounds (Schnittmenge der beiden Tabellen)
    bounds_gehalt = (max(praefmod1.loc[praefmod1['Attribut'] == 'Gehalt', 'RL_i'].iloc[0],
                          praefmod2.loc[praefmod2['Attribut'] == 'Gehalt', 'RL_i'].iloc[0]),
                     min(praefmod1.loc[praefmod1['Attribut'] == 'Gehalt', 'AL_i'].iloc[0],
                         praefmod2.loc[praefmod2['Attribut'] == 'Gehalt', 'AL_i'].iloc[0]))

    bounds_urlaub = (max(praefmod1.loc[praefmod1['Attribut'] == 'Urlaubstage', 'RL_i'].iloc[0],
                          praefmod2.loc[praefmod2['Attribut'] == 'Urlaubstage', 'RL_i'].iloc[0]),
                     min(praefmod1.loc[praefmod1['Attribut'] == 'Urlaubstage', 'AL_i'].iloc[0],
                         praefmod2.loc[praefmod2['Attribut'] == 'Urlaubstage', 'AL_i'].iloc[0]))

    bounds_zuzahlung = (max(praefmod1.loc[praefmod1['Attribut'] == 'Zuzahlung (PKW)', 'RL_i'].iloc[0],
                             praefmod2.loc[praefmod2['Attribut'] == 'Zuzahlung (PKW)', 'RL_i'].iloc[0]),
                        min(praefmod1.loc[praefmod1['Attribut'] == 'Zuzahlung (PKW)', 'AL_i'].iloc[0],
                            praefmod2.loc[praefmod2['Attribut'] == 'Zuzahlung (PKW)', 'AL_i'].iloc[0]))

    # Validierung der Bounds
    bounds = []
    for bound, name in zip([bounds_gehalt, bounds_urlaub, bounds_zuzahlung], ['Gehalt', 'Urlaubstage', 'Zuzahlung (PKW)']):
        if bound[0] > bound[1]:
            print(f"Warnung: Ungültige Schranken für {name}. Untergrenze ({bound[0]}) wird auf Obergrenze ({bound[1]}) gesetzt.")
            bounds.append((bound[1], bound[1]))  # Untergrenze wird auf Obergrenze gesetzt
        else:
            bounds.append(bound)

    initial_values = [angebot['Gehalt'], angebot['Urlaubstage'], angebot['Zuzahlung (PKW)']]
    result = minimize(zielfunktion, initial_values, bounds=bounds, method='SLSQP')

    max_nutzen = -result.fun

    return nutzen1, nutzen2, max_nutzen

def lade_dateien(change):
    try:
        global praefmod1, praefmod2
        print(upload1.value)
        print(upload2.value)
        memory_excel_1 = upload1.value[next(iter(upload1.value))]["content"]
        print(memory_excel_1)
        memory_excel_2 = upload2.value[next(iter(upload2.value))]["content"] 
        print(memory_excel_2)       
        praefmod1 = pd.read_excel(BytesIO(memory_excel_1), engine='openpyxl')
        praefmod2 = pd.read_excel(BytesIO(memory_excel_2), engine='openpyxl')
        #print(praefmod1.head())
        assert praefmod1['W_i'].sum() == 1.0, "Summe der Gewichtungen in Präfmod1 ist nicht 100%."
        assert praefmod2['W_i'].sum() == 1.0, "Summe der Gewichtungen in Präfmod2 ist nicht 100%."
        output.clear_output()
        with output:
            print("Dateien erfolgreich geladen.")
            display(praefmod1.head(), praefmod2.head())
    except Exception as e:
        with output:
            print("Fehler beim Laden der Dateien:", e)

def anzeigen_nutzenwerte(_):
    try:
        angebot = {k: float(v.value) for k, v in angebots_widgets.items()}
        nutzen1, nutzen2, max_nutzen = berechne_nutzenwerte(angebot, praefmod1, praefmod2)
        output.clear_output()
        with output:
            print(f"Nutzenwert Verhandlungspartner 1: {nutzen1:.3f}")
            print(f"Nutzenwert Verhandlungspartner 2: {nutzen2:.3f}")
            print(f"Maximaler Nutzenwert: {max_nutzen:.3f}")
    except Exception as e:
        with output:
            print("Fehler bei der Berechnung:", e)


# Widgets

# Upload Widgets
upload_layout = Layout(width='50%', margin='0 5px 0 0')  # Styling für die Upload-Widgets
upload1 = widgets.FileUpload(accept='.xlsx', multiple=False, layout=upload_layout)
upload2 = widgets.FileUpload(accept='.xlsx', multiple=False, layout=upload_layout)

# Lade Button
lade_button = widgets.Button(description="Excel-Dateien laden", icon='upload')
lade_button.style.button_color = 'lightblue'  # Farbe des Buttons
lade_button.layout = Layout(width='30%', padding='3px', margin='10px 0')
lade_button.on_click(lade_dateien)

# Angebots-Widgets
angebots_widgets = {
    'Gehalt': widgets.FloatText(description='Gehalt', value=5000.0),
    'Urlaubstage': widgets.FloatText(description='Urlaubstage', value=35.0),
    'Zuzahlung (PKW)': widgets.FloatText(description='Zuzahlung', value=250.0)
}

# Styling der Angebots-Widgets
for widget in angebots_widgets.values():
    widget.layout = Layout(width='50%', margin='5px 0')
    widget.style.description_width = '150px'  # Breite der Beschreibung

angebots_box = VBox(
    [widgets.Label("Angebot eingeben:", layout=Layout(margin='0 0 10px 0'))] + 
    list(angebots_widgets.values()),
    layout=Layout(
        border='1px solid lightgray',
        padding='15px',
        margin='15px 0',
        width='60%'
    )
)

# Berechnen Button
berechnen_button = widgets.Button(description="Nutzenwerte berechnen", icon='calculator')
berechnen_button.style.button_color = 'lightgreen'  # Farbe des Buttons
berechnen_button.layout = Layout(width='40%', padding='3px', margin='10px 0')
berechnen_button.on_click(anzeigen_nutzenwerte)

# Output Widget
output = Output()
output.layout = Layout(border='1px solid lightgray', padding='10px', width='80%')

# Finales UI Layout
ui = VBox([
    widgets.Label("Excel-Dateien hochladen:", layout=Layout(margin='0 0 10px 0')),
    HBox([upload1, upload2]),
    lade_button,
    angebots_box,
    berechnen_button,
    output
], layout=Layout(padding='20px', align_items='center'))  # Padding und zentrierte Inhalte

display(ui)

